In [2]:
import os
from collections import Counter
import glob
import numpy as np
from tqdm import tqdm
import yaml

import pdb

## Plot KITTI-360 vs Semantic-KITTI points 

In [17]:
from PIL import Image
import vispy.scene
from vispy.scene import visuals

semanticKitti_dir = 'data/SemanticKitti/sequences/08/velodyne/'
kitti360_dir = 'data/Kitti360/data_3d_raw/2013_05_28_drive_0002_sync/velodyne_points/data/'

semanticKitti_file = os.path.join(semanticKitti_dir, '000000.bin')
semanticKitti_points = np.fromfile(semanticKitti_file, dtype=np.float32).reshape(-1, 4)

In [30]:
canvas = vispy.scene.SceneCanvas(keys='interactive', show=True, size=(600,400), bgcolor='white')
grid = canvas.central_widget.add_grid()
scan_view = vispy.scene.widgets.ViewBox(border_color='white', parent=canvas.scene)
grid.add_widget(scan_view, 0, 0)
scan_vis = visuals.Markers()
scan_view.camera = 'turntable'
scan_view.add(scan_vis)
visuals.XYZAxis(parent=scan_view.scene)

scan_vis.set_data(semanticKitti_points[:, :3], edge_color=(1., 0., 0.), face_color=(1., 0., 0.), size=5)

In [31]:
vispy.app.run()
img_numpy = canvas.render()
img = Image.fromarray(img_numpy)
img.save('tmp.png')

AttributeError: 'NoneType' object has no attribute 'send_glir_commands'

## Plot point trajectories over frames

In [4]:
from utils.tracking_utils import *

In [10]:
dataset = '/project_data/ramanan/achakrav/4D-PLS/data/SemanticKitti/'

In [12]:
poses = []

calib = parse_calibration(os.path.join(dataset, "sequences", '08', "calib.txt"))
poses_f64 = parse_poses(os.path.join(dataset, "sequences", '08', "poses.txt"), calib)
poses.append([pose.astype(np.float32) for pose in poses_f64])

In [17]:
point_names = []
point_paths = os.path.join(dataset, "sequences", '08', "velodyne")
# populate the label names
seq_point_names = sorted(
    [os.path.join(point_paths, fn) for fn in os.listdir(point_paths) if fn.endswith(".bin")])

point_names.extend(seq_point_names)

idxs = np.array(len(point_names))

# Project the points w.r.t to the first frame
poses_seq = poses[0]
pose0 = poses_seq[0]

In [ ]:
for idx, point_file in enumerate(point_names):
    
    points = np.fromfile(point_file, dtype=np.float32).reshape([-1,4])
    hpoints = np.hstack((points[:, :3], np.ones_like(points[:, :1])))
    new_points = np.sum(np.expand_dims(hpoints, 2) * poses_seq[idx].T, axis=1)
    
    if idx == 0
    
    new_coords = new_points - pose0[:3, 3]
    # new_coords = new_coords.dot(pose0[:3, :3])
    new_coords = np.sum(np.expand_dims(new_coords, 2) * pose0[:3, :3], axis=1)
    new_coords = np.hstack((new_coords, points[, 3:]))


## Find frames with pedestrians

In [9]:
task_set = 0
result_dir = 'results/validation/TS{}/sequences/08/predictions/*.label'.format(task_set)
gt_dir = 'data/SemanticKitti/sequences/08/labels/*.label'

In [10]:
person_cls, car_cls = 30, 10
person_points, car_points = [], []
for file in tqdm(sorted(glob.glob(gt_dir))):
    labels = np.fromfile(file, dtype=np.int32)
    sem_label = labels & 0xFFFF
    person_points.append((sem_label == ped_cls).sum())
    car_points.append((sem_label == car_cls).sum())

100%|██████████| 4071/4071 [00:05<00:00, 751.90it/s]


#### Takeaways:
1. 1850-1900: decent number of person points (from 300 to 1000+ and then back down to 100)
2. No. of car points = 29898289, No. of person points = 79447

## Exploration on tracking

In [19]:
task_set = 1
ins_labels = Counter()
result_dir = 'test/val_preds_TS{}/val_probs/*_t.npy'.format(task_set)

In [20]:
for file in tqdm(sorted(glob.glob(result_dir))):
    labels = np.load(file)
    unique_labels = np.unique(labels)
    for label in unique_labels:
        if label > 0:
            if len(np.where(labels == label)[0]) < 25:
                continue
            ins_labels[label] += 1
#     ins_labels = np.unique(np.concatenate([ins_labels, unique_labels]))

100%|██████████| 2675/2675 [00:14<00:00, 184.52it/s]


In [26]:
ins_labels.keys()

dict_keys([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 22.0, 23.0, 24.0, 32.0, 35.0, 38.0, 41.0, 45.0, 47.0, 49.0, 50.0, 25.0, 27.0, 28.0, 31.0, 33.0, 34.0, 37.0, 39.0, 40.0, 42.0, 43.0, 44.0, 54.0, 55.0, 56.0, 57.0, 61.0, 63.0, 65.0, 66.0, 70.0, 72.0, 75.0, 76.0, 82.0, 89.0, 26.0, 29.0, 48.0, 51.0, 53.0, 58.0, 91.0, 94.0, 95.0, 96.0, 114.0, 21.0, 30.0, 36.0, 73.0, 79.0, 46.0, 52.0, 60.0, 64.0, 80.0, 59.0, 62.0, 67.0, 69.0, 83.0, 78.0, 71.0, 74.0, 85.0, 86.0, 87.0, 81.0, 84.0, 68.0, 77.0, 93.0, 97.0, 92.0, 88.0, 90.0, 102.0, 98.0, 103.0, 100.0, 104.0, 101.0, 99.0, 109.0, 108.0, 110.0, 105.0, 107.0, 111.0, 116.0, 106.0, 112.0])

In [21]:
task_set = 1
ins_labels_new = Counter()
result_dir = 'results/validation/TS{}/sequences/08/predictions/*.label'.format(task_set)

In [22]:
for file in tqdm(sorted(glob.glob(result_dir))):
    labels = np.fromfile(file, dtype=np.uint32)
    ins_label = labels >> 16
    unique_labels = np.unique(ins_label)
    for label in unique_labels:
        if label > 0:
            ins_labels_new[label] += 1
#     ins_labels = np.unique(np.concatenate([ins_labels, unique_labels]))

100%|██████████| 2675/2675 [00:04<00:00, 555.02it/s]


In [27]:
ins_labels_new.keys()

dict_keys([59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 29, 30, 41, 46, 48, 52, 53, 58, 91, 92, 94, 95, 96, 97, 99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 110, 25, 27, 42, 49, 54, 55, 83, 115, 116, 131, 134, 82, 84, 85, 86, 87, 88, 89, 90, 93, 98, 108, 111, 79, 80, 81, 112, 113, 114, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 132, 133, 21, 135, 136, 137, 50, 51, 56, 57, 45, 47, 40, 43, 44, 39, 35, 36, 37, 38, 33, 34, 31, 32, 28, 20, 22, 23, 24, 26])

In [16]:
np.sort(list(ins_labels.values())) == np.sort(list(ins_labels_new.values()))

/opt/anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

## Visualization video

In [2]:
task_set = 1
result_dir = 'test/val_preds_TS{}/val_probs/*'.format(task_set)
sem_labels = []

In [ ]:
non_semantic_ext = ['_c', '_i', '_u', '.ply', '_e']
for file in sorted(glob.glob(result_dir)):
    if any(ext in file for ext in non_semantic_ext):
        continue

    labels = np.load(file)
    unique_labels = np.unique(labels)
    sem_labels = np.unique(np.concatenate([sem_labels, unique_labels]))

In [ ]:
sem_labels

In [11]:
data_cfg = 'data/SemanticKitti/semantic-kitti.yaml'
label_dir = 'data/SemanticKitti/sequences/08/labels/*'
gt_labels = []

DATA = yaml.safe_load(open(data_cfg, 'r'))
class_map = DATA["task_set_map"][task_set]["learning_map"]

learning_map = np.zeros((np.max([k for k in class_map.keys()]) + 1), dtype=np.int32)
for k, v in class_map.items():
    learning_map[k] = v

In [12]:
for file in sorted(glob.glob(label_dir)):
    if '.npy' in file:
        continue
    
    labels = np.fromfile(file, dtype=np.uint32)
    sem_gt = labels & 0xFFFF
    unique_labels = np.unique(learning_map[sem_gt])
    gt_labels = np.unique(np.concatenate([gt_labels, unique_labels]))

In [1]:
gt_labels

NameError: name 'gt_labels' is not defined

In [4]:
from IPython.display import Video
Video("valid_vis/gt.mp4")